In [ ]:
!pip install spacy==3.0.6

In [ ]:
import pandas as pd
import glob
import json

In [ ]:
data_path = '/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data'

### Sentenze Penali Cassazione
http://www.italgiure.giustizia.it/sncass/ 

In [ ]:
pdf1 = pd.read_csv(data_path +'/sentenze_penale_pdf.csv')
pdf2 = pd.read_csv(data_path +'/sentenze_penale_pdf2.csv')

In [ ]:
pdfs = pd.concat([pdf1,pdf2])

In [ ]:
pdfs

In [ ]:
pdfs['text'].nunique()

In [ ]:
pdfs['chars'] = pdfs['text'].apply(len)
pdfs

In [ ]:
pdfs['chars'].plot.hist(bins=30)

In [ ]:
pdfs = pdfs[pdfs['chars']<40000]
pdfs

In [ ]:
pdfs[pdfs['chars'] <200]

In [ ]:
#Remove the lines that have a short text that correspond to judgments in the darkening phase
pdfs = pdfs[pdfs['chars'] > 200]

In [ ]:
pdfs

In [ ]:
pdfs['subtext'] = pdfs['text'].apply(lambda x: x.replace("'"," "))

pdfs['subtext'] = pdfs['subtext'].apply(lambda x: x[x.find(' 1.')+3: x.find(' 1.') + 500])

In [ ]:
pdfs.to_csv(data_path +'/sentenze_penali_clean.csv', index = False)

In [ ]:
df = pd.read_csv(data_path +'/sentenze_penali_clean.csv')
df

### Annotation

Use this web tool https://abera87.github.io/annotate/ from this paper https://arxiv.org/pdf/2108.08184.pdf

First sample

In [ ]:
sample_1 = df.head(26)
sample_1

In [ ]:
txt = " \n ".join(s.lower() for s in sample_1['subtext'])

In [ ]:
ann_file = open("/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/annotation_sample_1.txt", "w")
 
ann_file.write(txt)
 
ann_file.close()

In [ ]:
s1 = pd.read_json(data_path + '/Output_1.json').head(27)
s1

In [ ]:
s1 = s1[s1['EntityMentions'] != s1['RelationMentions'] ]
s1.reset_index(drop = True, inplace = True)
s1

Second sample

In [ ]:
df = pd.read_csv(data_path +'/sentenze_penali_clean.csv')
sample_2 = df[df.index >26].head(23)
sample_2

In [ ]:
txt = " \n ".join(s.lower() for s in sample_2['subtext'])

In [ ]:
ann_file = open("/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/annotation_sample_2.txt", "w")
 
ann_file.write(txt)
 
ann_file.close()

In [ ]:
s2 = pd.read_json(data_path + '/Output_2.json')
s2

thrid sample

In [ ]:
df = pd.read_csv(data_path +'/sentenze_penali_clean.csv')
sample_3 = df[df.index >49].head(50)
sample_3

In [ ]:
txt = " \n ".join(s.lower() for s in sample_3['subtext'])

In [ ]:
ann_file = open("/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/annotation_sample_3.txt", "w")
 
ann_file.write(txt)
 
ann_file.close()

In [ ]:
s3 = pd.read_json(data_path + '/Output_3.json')
s3

In [ ]:
s3 = s3[s3['EntityMentions'] != s3['RelationMentions'] ]
s3.reset_index(drop = True, inplace = True)
s3

Fourth Sample

In [ ]:
sample_4 = df[df.index >99]
sample_4

In [ ]:
txt = " \n ".join(s.lower() for s in sample_4['subtext'])

In [ ]:
ann_file = open("/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/annotation_sample_4.txt", "w")
 
ann_file.write(txt)
 
ann_file.close()

In [ ]:
s4 = pd.read_json(data_path + '/Output_4.json')
s4

In [ ]:
s4 = s4[s4['EntityMentions'] != s4['RelationMentions'] ]
s4.reset_index(drop = True, inplace = True)
s4

In [ ]:
s4.head(50)

In [ ]:
data_annotated = pd.concat([s1,s2,s3,s4])
data_annotated.reset_index(inplace = True,drop= True)
data_annotated

In [ ]:
data_annotated.drop('SentId',axis = 1, inplace = True)

In [ ]:
def remove_repetition(lista):
    lista_new = []
    for x in lista:
        contenuto = []
        for s in lista:
            if x != s:
                contenuto.append(x in s)
        if not (True in contenuto):
            lista_new.append(x)
    return lista_new

In [ ]:
data_annotated['EntityMentions'] = data_annotated['EntityMentions'].apply(remove_repetition)

In [ ]:
data_annotated

In [ ]:
#data_annotated = data_annotated.drop(index=23).reset_index(drop = True)
data_annotated

In [ ]:
data_annotated.iloc[20]['SentText']

In [ ]:
import re

def clean_jud(text):
    text = re.sub('art. \d', '',text)
    text = re.sub('artt. \d', '',text)
    text = re.sub("d.p.r \d", '',text)
    text = re.sub("n. \d", '',text)
    text = re.sub("n. dep \d", '',text)
    text = re.sub('ex \d','',text)

    text = text.replace('n.','')
    text = text.replace('cost.','')
    text = text.replace('sent.', '')
    text = text.replace('num.', '')
    text = text.replace('sez.', '')
    text = text.replace('p. e p.', '')
    text = text.replace("d.lgs.", '')
    text = text.replace("cod. pen.", '')
    text = text.replace("cod.proc.pe", '')
    text = text.replace('m.a.e', '')
    text = text.replace('lett.', '')
    

    
    text = re.sub(' +', ' ',text)
    return text


In [ ]:
data_annotated['SentText'] = data_annotated['SentText'].apply(clean_jud)

In [ ]:
data_annotated.iloc[2]['SentText']

In [ ]:
data_annotated.to_json(data_path +'/manually_annotated_dataset.json')

In [ ]:
data_annotated = pd.read_json(data_path +'/manually_annotated_dataset.json')

In [ ]:
data_annotated.iloc[0]['RelationMentions']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_annotated, dev_test = train_test_split(data_annotated, test_size=0.3)
dev_annotated, test_annotated = train_test_split(dev_test, test_size=0.5)

I have to convert this dataframe into a txt file

In [ ]:
import re
def pad(text):
    text = re.sub("([.,!?(')])", r' \1 ', text)
    text = re.sub('\s{2,}', ' ', text)
    return text

In [ ]:
def get_tokens(text):
    tokens = pad(text).split(' ')
    return tokens

In [ ]:
def nearest_entities(text, e1, e2):
    occ1 = [i for i in range(len(text)) if text.startswith(e1 , i)]
    occ2 = [i for i in range(len(text)) if text.startswith(e2, i)]
    
    minimum = len(text)
    indexes = (0,0)

    for i1,v1 in enumerate(occ1):
        for i2,v2 in enumerate(occ2):
            if abs(v1-v2) < minimum:
                minimum = abs(v1-v2)
                
                indexes = (i1,i2)
    tok_list = get_tokens(text)
    occ1_tokens = []
    for i,x in enumerate(tok_list):
        if len(get_tokens(e1)) == 1:
            if x == get_tokens(e1)[0]:
                occ1_tokens.append(i)
        elif len(get_tokens(e1)) == 2:
            if x == get_tokens(e1)[0] and tok_list[i+1] == get_tokens(e1)[1]:
                occ1_tokens.append(i)
        else:
            if x == get_tokens(e1)[0] and tok_list[i+1] == get_tokens(e1)[1] and tok_list[i+2] == get_tokens(e1)[2]:
                occ1_tokens.append(i)
    occ2_tokens = []
    for i,x in enumerate(tok_list):
        if len(get_tokens(e2)) == 1:
            if x == get_tokens(e2)[0]:
                occ2_tokens.append(i)
        elif len(get_tokens(e2)) == 2:
            if x == get_tokens(e2)[0] and tok_list[i+1] == get_tokens(e2)[1]:
                occ2_tokens.append(i)
        else:
            if x == get_tokens(e2)[0] and tok_list[i+1] == get_tokens(e2)[1] and tok_list[i+2] == get_tokens(e2)[2]:
                occ2_tokens.append(i)
    return occ1[indexes[0]], occ2[indexes[1]], occ1_tokens[indexes[0]], occ2_tokens[indexes[1]]

In [ ]:
def create_dict(df_row):
    tokens_list = get_tokens(df_row['SentText'])
    text = df_row['SentText']
    diz = {"document" : text,
           "tokens": [],
           "relations" : []}
    entities_list = []
    for rel in df_row['RelationMentions']:
        entity_1 = rel['Arg1Text']
        entity_2 = rel['Arg2Text']
        if entity_1 not in text or entity_2 not in text:
            continue
         
        tupla = nearest_entities(text, entity_1, entity_2)
        #print(tupla)


        if entity_1 not in df_row['EntityMentions']:
            continue
        ent1_diz = {"text": entity_1,
                                "start": tupla[0],
                                "end": tupla[0] + len(entity_1),
                                "token_start": tupla[2],
                                "token_end": tupla[2] + len(get_tokens(entity_1)) -1,
                                "entityLabel": rel['RelationNames'][0].split('/')[2].upper()}
        
        

        ent2_diz = {"text": entity_2,
                                "start": tupla[1],
                                "end": tupla[1] + len(entity_2),
                                "token_start": tupla[3],
                                "token_end": tupla[3] + len(get_tokens(entity_2)) -1,
                                "entityLabel": 'IMPUTATO'}
        diz['relations'].append({ "child": ent2_diz["token_start"], "head": ent1_diz["token_start"], "relationLabel": rel['RelationNames'][0].split('/')[1].upper() })
        if ent2_diz not in diz['tokens']:
            diz['tokens'].append(ent2_diz)
        if ent1_diz not in diz['tokens']:

            diz['tokens'].append(ent1_diz)
            
            

        
        
        
    return diz

In [ ]:
import json
def transform_json(df,filename):
    output = []
    for i in range(df.shape[0]):
        output.append(create_dict(df.iloc[i]))
    with open(data_path + f'/{filename}.txt', 'w') as fout:
        json.dump(output, fout)

In [ ]:
transform_json(data_annotated, 'prova')

In [ ]:
df = pd.read_json(data_path +'/prova.txt')

In [ ]:
df.iloc[44]['document']

In [ ]:
distances = []
for i in range(df.shape[0]):
    for diz in df.iloc[i]['relations']:
        m = abs(diz['head'] - diz['child'])
        distances.append(m)
print(distances)
print(max(distances))

In [ ]:
transform_json(train_annotated, 'relation_training')
transform_json(test_annotated, 'relation_test')
transform_json(dev_annotated, 'relation_dev')

In [ ]:
import json


from pathlib import Path

from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
#from spacy.lang.en import English
from spacy.util import compile_infix_regex
import re
import spacy

nlp = spacy.blank("it")
# Create a blank Tokenizer with just the English vocab

msg = Printer()

SYMM_LABELS = ["Binds"]
MAP_LABELS = {
    "DIFENDE": "DIFENDE",
    "GIUDICA": "GIUDICA"
}


def main(json_loc: Path, output_file: Path):
    """Creating the corpus from the Prodigy annotations."""
    Doc.set_extension("rel", default={},force=True)
    vocab = Vocab()

    docs = {"train": [], "dev": [], "test": [], "total": []}
    ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
    count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
    count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}

    with open(json_loc, encoding="utf8") as jsonfile:
        file = json.load(jsonfile)
        i = 0
        for example in file:
            span_starts = set()
            neg = 0
            pos = 0
                    # Parse the tokens
            tokens=nlp(example["document"])    

            spaces=[]
            spaces = [True if tok.whitespace_ else False for tok in tokens]
            words = [t.text for t in tokens]
            doc = Doc(nlp.vocab, words=words, spaces=spaces)


            # Parse the GGP entities
            spans = example["tokens"]
            entities = []
            span_end_to_start = {}
            for span in spans:
                entity = doc.char_span(
                     span["start"], span["end"], label=span["entityLabel"]
                 )
                

                span_end_to_start[span["token_start"]] = span["token_start"]
                #print(span_end_to_start)
                entities.append(entity)
                span_starts.add(span["token_start"])
            #print(entities)
            doc.ents = entities
            #print(i)
            i += 1

            # Parse the relations
            rels = {}
            for x1 in span_starts:
                for x2 in span_starts:
                    rels[(x1, x2)] = {}
                    #print(rels)
            relations = example["relations"]
            #print(len(relations))
            for relation in relations:
                # the 'head' and 'child' annotations refer to the end token in the span
                # but we want the first token
                start = span_end_to_start[relation["head"]]
                end = span_end_to_start[relation["child"]]
                label = relation["relationLabel"]
                #print(rels[(start, end)])
                #print(label)
                #label = MAP_LABELS[label]
                if label not in rels[(start, end)]:
                    rels[(start, end)][label] = 1.0
                    pos += 1
                    #print(pos)
                    #print(rels[(start, end)])

            # The annotation is complete, so fill in zero's where the data is missing
            for x1 in span_starts:
                for x2 in span_starts:
                    for label in MAP_LABELS.values():
                        if label not in rels[(x1, x2)]:
                            neg += 1
                            rels[(x1, x2)][label] = 0.0

                            #print(rels[(x1, x2)])
            doc._.rel = rels
            #print(doc._.rel)

            # only keeping documents with at least 1 positive case
            if pos > 0:
                    docs["total"].append(doc)
                    count_pos["total"] += pos
                    count_all["total"] += pos + neg

                    
                    
    #print(len(docs["total"]))
    docbin = DocBin(docs=docs["total"], store_user_data=True)
    docbin.to_disk(output_file)
    msg.info(
        f"{len(docs['total'])} training sentences"
    )
    #print(docs)

In [ ]:
train_file = data_path + '/train.spacy'
dev_file = data_path + '/dev.spacy'
test_file = data_path + '/test.spacy'

In [ ]:
main(data_path +'/relation_test.txt', test_file)

In [ ]:
main(data_path +'/relation_dev.txt', dev_file)

In [ ]:
main(data_path +'/relation_training.txt', train_file)

## Sentenze Giustizia Ammnistrativa
https://www.giustizia-amministrativa.it/web/guest/dcsnprr

In [ ]:
htmls = pd.read_csv(data_path +'/sentenze_html.csv')
htmls

In [ ]:
htmls['text'].nunique()